



Ce notebook implémente l'analyse sémantique avec un modèle de langage (LLM) pour organiser selon leur titre des documents .

#### Objectifs

- **Automatiser** la classification de documents médicaux (PDF, PPT, PPTX)
- **Identifier** automatiquement les clients, bases de données et domaines médicaux
- **Organiser** les fichiers en 4 catégories principales : CLIENTS, BASES_DONNEES, MALADIES, AUTRES
- **Générer** des rapports détaillés avec justifications sémantiques
- **Consolider** tous les fichiers pertinents dans un dossier global

#### Workflow du Processus

1. **Phase 1** : Recherche et filtrage des fichiers (extension, date, taille)
2. **Phase 2** : Copie vers le dossier de travail
3. **Phase 3** : Analyse sémantique avec LLM (GPT-4o-mini)
4. **Phase 4** : Classification et déplacement dans les dossiers
5. **Phase 5** : Consolidation dans `results_global`
6. **Phase 6** : Génération des rapports JSON et TXT
7. **Phase 7** : Dépôt SharePoint (optionnel)


Le système utilise **GPT-4o-mini** pour :
- Comprendre le **contexte** les titres de documents
- Détecter les **clients pharmaceutiques** (ABBVIE, IQVIA, etc.)
- Identifier les **bases de données** (Xponent, MIDAS, etc.)
- Reconnaître les **domaines médicaux** (oncologie, diabétologie, etc.)
- Fournir une **justification** pour chaque classification

#### Avantages par rapport aux méthodes classiques

- **Flexibilité** : S'adapte automatiquement aux nouveaux termes
- **Contexte** : Comprend l'intention derrière les titres
- **Traçabilité** : Justification de chaque décision
- **Précision** : Classification multi-critères intelligente

In [ ]:
pip install  langchain-openai langchain-core

In [ ]:
import io
from google.colab import files
uploaded = files.upload()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [83]:
#!/usr/bin/env python3


# ================================
# 📦 IMPORTS DES BIBLIOTHÈQUES
# ================================

import os
import shutil
import json
from datetime import datetime, timedelta
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate

print("✅ Imports réalisés avec succès")

# ================================
# 📁 CONFIGURATION DES CHEMINS
# ================================

# Configuration des chemins
BASE_DIR = os.path.abspath(r"C:\Users\kosmo\pycode\Iqvia_process")
CHEMIN_SOURCE = os.path.join(BASE_DIR, "ProcessEx")
CHEMIN_DEPOTS = os.path.join(BASE_DIR, "Depots")

print(f"📁 Dossier source: {CHEMIN_SOURCE}")
print(f"📁 Dossier dépôt: {CHEMIN_DEPOTS}")
print(f"✅ Chemins configurés")

# ================================
# 📄 LECTURE DU FICHIER PROMPT
# ================================

try:
    with open(
        os.path.join(BASE_DIR, "ProcessEx", "Prompt_depot_json_sharepoint.txt"),
        "r",
        encoding="utf-8",
    ) as file:
        prompt = file.read()
    print(f"✅ Fichier prompt lu avec succès, taille: {len(prompt)} caractères")
except FileNotFoundError:
    print("❌ Fichier Prompt_depot_json_sharepoint.txt non trouvé")
    prompt = ""
except Exception as e:
    print(f"❌ Erreur lors de la lecture du fichier: {e}")
    prompt = ""

# ================================
# 🧠 INITIALISATION DU LLM
# ================================

# Récupération de la clé API
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
   print("⚠️ ATTENTION: Clé API non trouvée dans les variables d'environnement")
   api_key = " 0yCT3BlbkFJ6nuNH0hRFwAy9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"  # À remplacer par votre vraie clé

# Initialiser le LLM
api_key = os.getenv('OPENAI_API_KEY')
if not api_key:
    print("⚠️ ATTENTION: Utilisez une variable d'environnement  ")
    api_key = " eoK0yCT3BlbkFJ6nuNH0hRFwAy9HhFfHS_cUMhXQMX6_U0pycw_XiZUUtZ4V6Gc5xEwhMZOsYA6xKN4HruNnPRcA"

llm = ChatOpenAI(
    model="gpt-4o-mini",
    temperature=0,
    api_key=api_key
)

✅ Imports réalisés avec succès
📁 Dossier source: C:\Users\kosmo\pycode\Iqvia_process\ProcessEx
📁 Dossier dépôt: C:\Users\kosmo\pycode\Iqvia_process\Depots
✅ Chemins configurés
✅ Fichier prompt lu avec succès, taille: 3008 caractères




#### Descriptif des fonctions par phases

##### PHASE EXTRACTION (Phases 1-2)

| Fonction | Description | Phase détaillée |
|----------|-------------|-----------------|
| `rechercher_fichiers_filtres()` | Recherche et filtre les fichiers par extension, date, taille | Phase 1 |
| `copier_fichiers()` | Copie les fichiers vers le dossier de travail | Phase 2 |

##### PHASE ANALYSE SÉMANTIQUE (Core + Phase 3)

| Fonction | Description | Phase détaillée |
|----------|-------------|-----------------|
| `analyser_titre_avec_llm_semantique()` | Analyse sémantique d'un titre avec GPT-4o-mini | Core |
| `analyser_et_classer_semantique()` | Classification automatique avec LLM | Phase 3 |

##### PHASE RAPPORT (Phases 4-7)

| Fonction | Description | Phase détaillée |
|----------|-------------|-----------------|
| `creer_dossiers_et_deplacer()` | Création des dossiers et déplacement des fichiers | Phase 4 |
| `consolider_resultats_globaux()` | Consolidation dans le dossier `results_global` | Phase 5 |
| `generer_rapport_final()` | Génération des rapports JSON et TXT | Phase 6 |
| `deposer_vers_sharepoint()` | Dépôt des fichiers vers SharePoint (optionnel) | Phase 7 |

---
 Fonction Clé : `analyser_titre_avec_llm_semantique()`
 
- **Input** : Titre de document (string)
- **Output** : Dictionnaire avec classification, clients détectés, domaines médicaux, justification

#### Flux global du processus
```
EXTRACTION → ANALYSE SÉMANTIQUE → RAPPORT
   ↓              ↓                 ↓
Phase 1-2      Core + Phase 3    Phase 4-7
```



In [84]:
# Définition des filtres
DATE_LIMITE = datetime.now() - timedelta(days=365)  # Fichiers de moins d'un an
TAILLE_CLASSIFICATION = (
        2000 * 1024
)  # MODIFIEZ CETTE VALEUR pour changer la taille minimum


In [85]:
def rechercher_fichiers_filtres():
    """Phase 1: Recherche et filtre les fichiers par extension, date et taille"""
    print("🔍 PHASE 1: Recherche et filtrage des fichiers...")
    print("-" * 50)

    if not os.path.exists(CHEMIN_SOURCE):
        print(f"❌ Erreur: {CHEMIN_SOURCE} n'existe pas")
        return []

    
    print(
        f"📅 Filtre date: fichiers modifiés après le {DATE_LIMITE.strftime('%d/%m/%Y')}"
    )
    print(f"📏 Filtre taille: minimum {TAILLE_CLASSIFICATION//1024} KB")
    print(f"📁 Seuls les fichiers > {TAILLE_CLASSIFICATION//1024} KB sont retenus")
    print()

    fichiers_trouves = []
    dossiers_stats = {}
    stats_filtrage = {
        "total_examines": 0,
        "rejetes_extension": 0,
        "rejetes_taille": 0,
        "rejetes_date": 0,
        "rejetes_acces": 0,
        "acceptes": 0,
    }

    for item in os.listdir(CHEMIN_SOURCE):
        item_path = os.path.join(CHEMIN_SOURCE, item)
        if os.path.isdir(item_path):
            print(f"📂 Analyse du dossier: {item}")
            count_dossier = 0

            for root, dirs, files in os.walk(item_path):
                for file in files:
                    file_path = os.path.join(root, file)
                    stats_filtrage["total_examines"] += 1

                    # Vérification de l'extension
                    if not file.lower().endswith((".ppt", ".pptx", ".pdf")):
                        stats_filtrage["rejetes_extension"] += 1
                        continue

                    try:
                        # Récupération des métadonnées
                        taille = os.path.getsize(file_path)
                        date_mod = datetime.fromtimestamp(os.path.getmtime(file_path))

                        # Filtrage par taille - SEUL FILTRE DE TAILLE
                        if taille < TAILLE_CLASSIFICATION:
                            stats_filtrage["rejetes_taille"] += 1
                            continue

                        # Filtrage par date
                        if date_mod < DATE_LIMITE:
                            stats_filtrage["rejetes_date"] += 1
                            continue

                        # Fichier accepté
                        fichier_info = {
                            "chemin": file_path,
                            "nom": file,
                            "taille": taille,
                            "date_modification": date_mod,
                            "taille_mb": round(taille / (1024 * 1024), 2),
                            "taille_kb": round(taille / 1024, 1),
                            "age_jours": (datetime.now() - date_mod).days,
                            "classification_semantique": True,  # Tous les fichiers retenus sont > TAILLE_CLASSIFICATION
                        }

                        fichiers_trouves.append(fichier_info)
                        count_dossier += 1
                        stats_filtrage["acceptes"] += 1

                    except (OSError, PermissionError) as e:
                        stats_filtrage["rejetes_acces"] += 1
                        print(
                            f"   ⚠️ Ignoré (accès refusé): {os.path.basename(file_path)}"
                        )

            dossiers_stats[item] = count_dossier
            print(f"   ✅ {count_dossier} fichier(s) retenu(s) après filtrage")

    # Affichage des statistiques détaillées
    print(f"\n📊 STATISTIQUES DE FILTRAGE:")
    print(f"   • Fichiers examinés: {stats_filtrage['total_examines']}")
    print(f"   • ❌ Rejetés - extension: {stats_filtrage['rejetes_extension']}")
    print(f"   • ❌ Rejetés - taille: {stats_filtrage['rejetes_taille']}")
    print(f"   • ❌ Rejetés - date: {stats_filtrage['rejetes_date']}")
    print(f"   • ❌ Rejetés - accès: {stats_filtrage['rejetes_acces']}")
    print(f"   • ✅ RETENUS: {stats_filtrage['acceptes']}")

    print(f"\n📁 RÉPARTITION PAR DOSSIER:")
    for dossier, count in dossiers_stats.items():
        print(f"   • {dossier}: {count} fichier(s)")

    return fichiers_trouves

In [ ]:
fichiers_info = rechercher_fichiers_filtres()

### Fonction `copier_fichiers`

#### Description
Copie une liste de fichiers vers le dossier `Depots` avec affichage des informations de progression.

1. **Création du dossier** : Crée le dossier `CHEMIN_DEPOTS` s'il n'existe pas
2. **Copie des fichiers** : Itère sur chaque fichier et le copie vers la destination
3. **Gestion d'erreurs** : Affiche les erreurs de copie sans interrompre le processus
4. **Statistiques** : Affiche le nombre de fichiers copiés et la taille totale



In [87]:
def copier_fichiers(fichiers_info):
    """Phase 2: Copie vers Depots avec informations enrichies"""
    print(f"\n📋 PHASE 2: Copie vers {CHEMIN_DEPOTS}...")
    print("-" * 50)

    # Créer le dossier Depots
    os.makedirs(CHEMIN_DEPOTS, exist_ok=True)
    print(f"✅ Dossier Depots prêt")

    copied = 0
    taille_totale = 0

    for fichier_info in fichiers_info:
        file_path = fichier_info["chemin"]
        file_name = fichier_info["nom"]
        destination = os.path.join(CHEMIN_DEPOTS, file_name)

        try:
            shutil.copy(file_path, destination)
            copied += 1
            taille_totale += fichier_info["taille"]
        except Exception as e:
            print(f"❌ Erreur copie {file_name}: {e}")

    print(f"✅ {copied}/{len(fichiers_info)} fichiers copiés")
    print(f"📦 Taille totale copiée: {taille_totale/(1024*1024):.1f} MB")
    return copied

In [ ]:
copied = copier_fichiers(fichiers_info)

### Fonction `analyser_titre_avec_llm_semantique`

#### Description
Analyse le titre d'un document avec un LLM pour extraire automatiquement les informations métier (client, maladie, base de données, type de document).

 
#### Valeur de retour
Dictionnaire contenant :
- `contexte_principal` : "Data" ou "autre"
- `domaine_medical` : Domaine médical détecté ou "null"
- `type_document` : Type de document (suivi, rapport, étude, etc.)
- `clients_detectes` : Nom du client principal ou "null"
- `bases_detectees` : Base de données détectée ou "null"
- `contient_maladie` : Boolean
- `confiance` : "haute", "moyenne" ou "faible"
- `score_semantique` : Score de 0.1 à 0.95
- `categorie_medicale` : Catégorie pour le classement

#### Fonctionnement

1. **Vérification LLM** : Vérifie si le LLM est disponible
2. **Analyse sémantique** : Envoie un prompt détaillé au LLM pour analyser le contexte
3. **Classification** : Détermine la catégorie selon la priorité :
   - `domaine_medical` si maladie détectée
   - `base_donnees` si base détectée  
   - `client` si client détecté
   - `autres` sinon
4. **Calcul du score** : Évalue la qualité de l'analyse

 

In [ ]:
import json
import os
import sys  # Ajout de la bibliothèque sys
from datetime import datetime, timedelta

def analyser_titre_avec_llm_semantique(titre: str) -> dict:

        """Analyse sémantique avancée avec LLM - prompt enrichi"""
        if not llm:
            print(f"❌ LLM non disponible, impossible d'analyser '{titre[:30]}...'")
            return {
                "contient_maladie": False,
                "maladies_detectees": [],
                "categorie_medicale": "autres",
                "contexte_principal": "autre",
                "clients_detectes": [],
                "bases_detectees": [],
                "confiance": "faible",
                "titre_normalise": titre,
                "score_semantique": 0.1,
            }

        try:
            analysis_prompt_template = ChatPromptTemplate.from_messages(
                [
                    (
                        "system",
                        """Tu es un expert en analyse sémantique médicale et business pharmaceutique.
                Ton rôle est de comprendre le CONTEXTE et l'INTENTION derrière chaque titre, pas seulement chercher des mots-clés.


                """,
                    ),
                    (
                        "human",
                        prompt,
                    ),
                ]
            )
            
            prompt_value = analysis_prompt_template.invoke({"titre": titre})
            response = llm.invoke(prompt_value.to_messages())
            content = response.content.strip()

            # Nettoyer le JSON
            if "```json" in content:
                content = content.split("```json")[1].split("```")[0]
            elif "```" in content:
                content = content.split("```")[1].split("```")[0]

            resultat = json.loads(content.strip())
            print(f"🔍 DEBUG resultat: {resultat}")

            # Validation et enrichissement
            if "categorie_medicale" not in resultat:
                resultat["categorie_medicale"] = "autres"  # Fallback par défaut

            # NOUVELLE logique simplifiée pour 4 dossiers - ORDRE CORRIGÉ
            if (
                resultat.get("domaine_medical")
                and resultat.get("domaine_medical") != "null"
            ):
                resultat["categorie_medicale"] = "domaine_medical"
            elif (
                resultat.get("bases_detectees")
                and resultat.get("bases_detectees") != "null"
            ):
                resultat["categorie_medicale"] = "base_donnees"
            elif resultat.get("clients_detectes") and resultat.get(
                "clients_detectes"
            ) not in ["null", "aucun"]:
                resultat["categorie_medicale"] = "client"
            else:
                resultat["categorie_medicale"] = "autres"

            # Calculer score sémantique si absent
            if "score_semantique" not in resultat or not isinstance(
                resultat["score_semantique"], (int, float)
            ):
                contexte = resultat.get("contexte_principal", "autre")
                domaine = resultat.get("domaine_medical", "aucun")

                score = 0.3  # Base
                if contexte != "autre":
                    score += 0.2
                if domaine != "aucun":
                    score += 0.2
                if len(resultat.get("maladies_detectees", [])) > 0:
                    score += 0.2
                if resultat.get("confiance") == "haute":
                    score += 0.1

                resultat["score_semantique"] = min(0.95, score)

            return resultat

        except Exception as e:
            print(f"❌ Erreur LLM sémantique pour '{titre[:30]}...': {e}")
            return {
                "contient_maladie": False,
                "maladies_detectees": [],
                "categorie_medicale": "autres",
                "contexte_principal": "autre",
                "clients_detectes": [],
                "bases_detectees": [],
                "confiance": "faible",
                "titre_normalise": titre,
                "score_semantique": 0.1,
            }

### Fonction `analyser_et_classer_semantique`

#### Description
Analyse et classe automatiquement les documents selon leur taille : analyse sémantique LLM pour les gros fichiers (> 2MB), classement automatique dans "AUTRES" pour les petits fichiers.

 

In [90]:
def analyser_et_classer_semantique():
    """Phase 3: Analyse sémantique et classification (sans filtrage redondant)"""
    print(f"\n🧠 PHASE 3: Analyse sémantique et classification...")
    print("-" * 50)

    # Mode LLM ou fallback
    use_llm = llm is not None
    print(f"Mode: {'🔥 LLM Sémantique' if use_llm else '🔧 Mots-clés enrichis'}")

    fichiers_a_analyser = [
        f
        for f in os.listdir(CHEMIN_DEPOTS)
        if f.lower().endswith((".ppt", ".pptx", ".pdf"))
    ]

    analyses = []
    medicaux = 0
    contextes_stats = {}

    print(f"📊 Fichiers à analyser: {len(fichiers_a_analyser)}")
    print()

    for i, file_name in enumerate(fichiers_a_analyser, 1):
        titre = os.path.splitext(file_name)[0]

        # Récupérer la taille pour affichage (optionnel)
        try:
            file_path = os.path.join(CHEMIN_DEPOTS, file_name)
            taille = os.path.getsize(file_path)
            taille_mb = round(taille / (1024 * 1024), 2)
            taille_kb = round(taille / 1024, 1)
            taille_str = f"{taille_kb} KB" if taille_mb < 1 else f"{taille_mb} MB"
        except:
            taille_str = "Taille inconnue"
            taille_mb = 0

        print(f"[{i:2d}/{len(fichiers_a_analyser)}] {file_name[:40]}... ({taille_str})")

        # Classification sémantique directe (plus de filtrage)
        analyse = analyser_titre_avec_llm_semantique(titre)

        # Enrichir avec métadonnées
        analyse["nom_fichier"] = file_name
        analyse["taille_mb"] = taille_mb

        # Compter et afficher résultats
        if analyse.get("contient_maladie", False):
            medicaux += 1

        contexte = analyse.get("contexte_principal", "autre")
        contextes_stats[contexte] = contextes_stats.get(contexte, 0) + 1

        # Affichage du résultat
        categorie = analyse.get("categorie_medicale", "aucune")
        print(f"🔍 DEBUG !!!!!!!!!!!!!!!!!!!!! categorie: {categorie}")

        confiance = analyse.get("confiance", "inconnue")
        score = analyse.get("score_semantique", 0)

        if analyse.get("clients_detectes"):
            client_info = f" [Client: {', '.join(analyse['clients_detectes'])}]"
        elif analyse.get("bases_detectees"):
            client_info = f" [Base: {', '.join(analyse['bases_detectees'])}]"
        else:
            client_info = ""

        print(
            f"   🎯 {categorie.upper()} ({confiance}, score:{score:.2f}){client_info}"
        )

        if analyse.get("maladies_detectees"):
            print(f"   🏥 Termes: {', '.join(analyse['maladies_detectees'])}")

        analyses.append(analyse)

    print(f"\n📊 RÉSULTATS DE CLASSIFICATION:")
    print(f"   • Fichiers analysés: {len(analyses)}")
    print(f"   • Fichiers médicaux détectés: {medicaux}")
    print(
        f"   • Score sémantique moyen: {sum(a.get('score_semantique', 0) for a in analyses) / len(analyses):.2f}"
    )

    print(f"\n📈 RÉPARTITION PAR CONTEXTE:")
    for contexte, count in sorted(contextes_stats.items()):
        pct = (count / len(analyses) * 100) if analyses else 0
        print(f"   • {contexte}: {count} fichiers ({pct:.1f}%)")

    return analyses

In [ ]:
analyses = analyser_et_classer_semantique()

### Fonction `creer_dossiers_et_deplacer`

#### Description
Crée les dossiers de classification et déplace les fichiers analysés dans les bonnes catégories selon leur analyse sémantique.



In [92]:
def creer_dossiers_et_deplacer(analyses):
    """Phase 4: Création des dossiers de classification et déplacement"""
    print(f"\n📁 PHASE 4: Création dossiers et déplacement...")
    print("-" * 50)

    # Mapping simplifié pour 4 dossiers
    mapping_dossiers = {
        "client": "CLIENTS",
        "base_donnees": "BASES_DONNEES",
        "domaine_medical": "MALADIES",  # ← MAINTENANT ÇA MARCHE !
        "autres": "AUTRES",
    }

    # Créer tous les dossiers nécessaires
    dossiers_crees = set()
    for categorie in mapping_dossiers.values():
        dossier_path = os.path.join(CHEMIN_DEPOTS, categorie)
        os.makedirs(dossier_path, exist_ok=True)
        dossiers_crees.add(categorie)

    print(f"✅ {len(dossiers_crees)} dossiers de classification créés")

    # Déplacer les fichiers
    stats_deplacement = {}
    erreurs_deplacement = []

    for analyse in analyses:
        file_name = analyse["nom_fichier"]
        categorie = analyse.get("categorie_medicale", "autres")
        dossier_cible = mapping_dossiers.get(categorie, "AUTRES")

        source_path = os.path.join(CHEMIN_DEPOTS, file_name)
        destination_path = os.path.join(CHEMIN_DEPOTS, dossier_cible, file_name)

        try:
            if os.path.exists(source_path):
                # Gérer les doublons
                if os.path.exists(destination_path):
                    base, ext = os.path.splitext(file_name)
                    counter = 1
                    while os.path.exists(destination_path):
                        new_name = f"{base}_({counter}){ext}"
                        destination_path = os.path.join(
                            CHEMIN_DEPOTS, dossier_cible, new_name
                        )
                        counter += 1

                shutil.move(source_path, destination_path)
                stats_deplacement[dossier_cible] = (
                    stats_deplacement.get(dossier_cible, 0) + 1
                )

        except Exception as e:
            erreurs_deplacement.append(f"{file_name}: {str(e)}")
            print(f"❌ Erreur déplacement {file_name}: {e}")

    print(f"\n📊 RÉSULTATS DE DÉPLACEMENT:")
    total_deplaces = sum(stats_deplacement.values())
    for dossier, count in sorted(stats_deplacement.items()):
        pct = (count / total_deplaces * 100) if total_deplaces > 0 else 0
        print(f"   • {dossier}: {count} fichiers ({pct:.1f}%)")

    if erreurs_deplacement:
        print(f"\n❌ ERREURS DE DÉPLACEMENT ({len(erreurs_deplacement)}):")
        for erreur in erreurs_deplacement[:5]:  # Limiter à 5 erreurs
            print(f"   • {erreur}")
        if len(erreurs_deplacement) > 5:
            print(f"   • ... et {len(erreurs_deplacement) - 5} autres erreurs")

    return stats_deplacement

In [ ]:
stats_deplacement = creer_dossiers_et_deplacer(analyses)

### Fonction `consolider_resultats_globaux`

#### Description
Copie tous les fichiers classés des différents dossiers vers un dossier unique `results_global` pour faciliter l'accès et la consultation.



In [94]:
def consolider_resultats_globaux():
    """Phase 6: Consolidation de tous les fichiers classés dans un dossier global"""
    print(f"\n📁 PHASE 6: Consolidation dans results_global...")
    print("-" * 50)

    # Créer le dossier results_global
    results_path = os.path.join(CHEMIN_DEPOTS, "results_global")
    os.makedirs(results_path, exist_ok=True)

    # Liste des dossiers à consolider (tous sauf results_global)
    dossiers_a_consolider = [
        "CLIENTS",
        "BASES_DONNEES",
        "MALADIES",  # ← 3 dossiers seulement
    ]

    fichiers_copies = 0
    erreurs_copie = []

    for dossier in dossiers_a_consolider:
        dossier_path = os.path.join(CHEMIN_DEPOTS, dossier)

        if os.path.exists(dossier_path):
            print(f"📂 Consolidation de {dossier}...")

            for file_name in os.listdir(dossier_path):
                if file_name.lower().endswith((".ppt", ".pptx", ".pdf")):
                    source = os.path.join(dossier_path, file_name)
                    destination = os.path.join(results_path, file_name)

                    try:
                        # Gérer les doublons avec préfixe du dossier source
                        if os.path.exists(destination):
                            name, ext = os.path.splitext(file_name)
                            new_name = f"{dossier}_{name}{ext}"
                            destination = os.path.join(results_path, new_name)

                        shutil.copy2(
                            source, destination
                        )  # copy2 préserve les métadonnées
                        fichiers_copies += 1

                    except Exception as e:
                        erreurs_copie.append(f"{dossier}/{file_name}: {str(e)}")

    print(f"\n✅ Consolidation terminée:")
    print(f"   • {fichiers_copies} fichiers copiés dans {results_path}")
    print(
        f"   • {len(erreurs_copie)} erreurs" if erreurs_copie else "   • Aucune erreur"
    )

    if erreurs_copie:
        print(f"\n❌ ERREURS DE COPIE:")
        for erreur in erreurs_copie[:3]:
            print(f"   • {erreur}")
        if len(erreurs_copie) > 3:
            print(f"   • ... et {len(erreurs_copie) - 3} autres erreurs")

    return fichiers_copies

In [ ]:
fichiers_consolides = consolider_resultats_globaux()

### Fonction `generer_rapport_final`

#### Description
Génère un rapport détaillé de l'ensemble du processus de classification sous deux formats : JSON (données structurées) et TXT (rapport lisible).



In [96]:
def generer_rapport_final(analyses, stats_deplacement):
    """Phase 5: Génération du rapport final détaillé"""
    print(f"\n📋 PHASE 5: Génération du rapport final...")
    print("-" * 50)

    TAILLE_CLASSIFICATION_MB = 2700 / 1024  # Convertir 1000 KB en MB = 0.98 MB

    # Créer le rapport JSON détaillé
    rapport = {
        "metadata": {
            "date_execution": datetime.now().isoformat(),
            "version_script": "2.0",
            "mode_llm": llm is not None,
            "total_fichiers": len(analyses),
        },
        "statistiques_globales": {
            "fichiers_medicaux": sum(
                1 for a in analyses if a.get("contient_maladie", False)
            ),
            "fichiers_clients": sum(1 for a in analyses if a.get("clients_detectes")),
            "fichiers_bases_donnees": sum(
                1 for a in analyses if a.get("bases_detectees")
            ),
            "score_semantique_moyen": (
                round(
                    sum(a.get("score_semantique", 0) for a in analyses) / len(analyses),
                    3,
                )
                if analyses
                else 0
            ),
        },
        "repartition_dossiers": stats_deplacement,
        "analyses_detaillees": [
            a for a in analyses if a.get("taille_mb", 0) >= TAILLE_CLASSIFICATION_MB
        ],
    }

    # Sauvegarder le rapport JSON
    rapport_path = os.path.join(CHEMIN_DEPOTS, "rapport_classification.json")
    try:
        with open(rapport_path, "w", encoding="utf-8") as f:
            json.dump(rapport, f, indent=2, ensure_ascii=False, default=str)
        print(f"✅ Rapport JSON sauvegardé: {rapport_path}")
    except Exception as e:
        print(f"❌ Erreur sauvegarde rapport: {e}")

    # Générer rapport texte lisible
    rapport_txt_path = os.path.join(CHEMIN_DEPOTS, "rapport_classification.txt")
    try:
        with open(rapport_txt_path, "w", encoding="utf-8") as f:
            f.write("=" * 80 + "\n")
            f.write("RAPPORT DE CLASSIFICATION MÉDICALE AUTOMATIQUE\n")
            f.write("=" * 80 + "\n\n")

            f.write(
                f"📅 Date d'exécution: {datetime.now().strftime('%d/%m/%Y %H:%M:%S')}\n"
            )
            f.write(f"🔧 Mode: {'LLM Sémantique' if llm else 'Mots-clés enrichis'}\n")
            f.write(f"📁 Dossier source: {CHEMIN_SOURCE}\n")
            f.write(f"📁 Dossier dépôt: {CHEMIN_DEPOTS}\n\n")

            f.write("📊 STATISTIQUES GLOBALES\n")
            f.write("-" * 40 + "\n")
            f.write(f"Total fichiers traités: {len(analyses)}\n")
            f.write(
                f"Fichiers médicaux: {rapport['statistiques_globales']['fichiers_medicaux']}\n"
            )
            f.write(
                f"Fichiers clients: {rapport['statistiques_globales']['fichiers_clients']}\n"
            )
            f.write(
                f"Fichiers bases données: {rapport['statistiques_globales']['fichiers_bases_donnees']}\n"
            )
            f.write(
                f"Score sémantique moyen: {rapport['statistiques_globales']['score_semantique_moyen']}\n\n"
            )

            f.write("📁 RÉPARTITION PAR DOSSIERS\n")
            f.write("-" * 40 + "\n")
            for dossier, count in sorted(stats_deplacement.items()):
                pct = (count / len(analyses) * 100) if analyses else 0
                f.write(f"{dossier}: {count} fichiers ({pct:.1f}%)\n")

            # Comptage du dossier results_global
            results_global_path = os.path.join(CHEMIN_DEPOTS, "results_global")
            if os.path.exists(results_global_path):
                fichiers_consolides = len(
                    [
                        f
                        for f in os.listdir(results_global_path)
                        if f.lower().endswith((".ppt", ".pptx", ".pdf"))
                    ]
                )
                f.write(f"\n📂 CONSOLIDATION GLOBALE\n")
                f.write("-" * 40 + "\n")
                f.write(f"RESULTS_GLOBAL: {fichiers_consolides} fichiers consolidés\n")

        print(f"✅ Rapport texte sauvegardé: {rapport_txt_path}")
    except Exception as e:
        print(f"❌ Erreur sauvegarde rapport texte: {e}")

    return rapport

In [ ]:
rapport = generer_rapport_final(analyses, stats_deplacement)

In [ ]:
########################################################################################################

In [ ]:
#def deposer_vers_sharepoint():
    """Phase 7: Dépôt du dossier results_global vers SharePoint"""
    print(f"\n☁️ PHASE 7: Dépôt vers SharePoint...")
    print("-" * 50)

    try:
        from office365.runtime.auth.authentication_context import AuthenticationContext
        from office365.sharepoint.client_context import ClientContext
        import requests

        # Configuration SharePoint (à adapter selon vos paramètres)
        SHAREPOINT_URL = "https://votreentreprise.sharepoint.com/sites/votre-site"
        SHAREPOINT_USERNAME = os.getenv("SHAREPOINT_USERNAME")
        SHAREPOINT_PASSWORD = os.getenv("SHAREPOINT_PASSWORD")
        SHAREPOINT_LIBRARY = (
            "Documents Partagés/Classification_Medicale"  # Dossier de destination
        )

        # Vérifier les paramètres
        if not all([SHAREPOINT_USERNAME, SHAREPOINT_PASSWORD]):
            print("❌ Erreur: Variables d'environnement SharePoint manquantes")
            print("   Définissez SHAREPOINT_USERNAME et SHAREPOINT_PASSWORD")
            return False

        # Chemin du dossier local à uploader
        results_path = os.path.join(CHEMIN_DEPOTS, "results_global")
        if not os.path.exists(results_path):
            print(f"❌ Erreur: Dossier {results_path} n'existe pas")
            return False

        # Authentification SharePoint
        print("🔐 Authentification SharePoint...")
        auth_context = AuthenticationContext(url=SHAREPOINT_URL)
        auth_context.acquire_token_for_user(
            username=SHAREPOINT_USERNAME, password=SHAREPOINT_PASSWORD
        )

        ctx = ClientContext(SHAREPOINT_URL, auth_context)
        web = ctx.web
        ctx.load(web)
        ctx.execute_query()

        print(f"✅ Connecté à SharePoint: {web.properties['Title']}")

        # Créer le dossier de destination si nécessaire
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        destination_folder = f"{SHAREPOINT_LIBRARY}/Classification_{timestamp}"

        # Upload des fichiers
        fichiers_uploades = 0
        erreurs_upload = []

        print(f"📂 Upload vers: {destination_folder}")

        for file_name in os.listdir(results_path):
            if file_name.lower().endswith((".ppt", ".pptx", ".pdf")):
                file_path = os.path.join(results_path, file_name)

                try:
                    with open(file_path, "rb") as file_content:
                        # Upload du fichier
                        target_folder = ctx.web.get_folder_by_server_relative_url(
                            destination_folder
                        )
                        target_folder.upload_file(file_name, file_content.read())
                        ctx.execute_query()

                        fichiers_uploades += 1
                        print(f"   ✅ {file_name}")

                except Exception as e:
                    erreurs_upload.append(f"{file_name}: {str(e)}")
                    print(f"   ❌ {file_name}: {e}")

        print(f"\n📊 RÉSULTATS UPLOAD SHAREPOINT:")
        print(f"   • {fichiers_uploades} fichiers uploadés")
        print(f"   • {len(erreurs_upload)} erreurs")
        print(f"   • Destination: {destination_folder}")

        if erreurs_upload:
            print(f"\n❌ ERREURS D'UPLOAD:")
            for erreur in erreurs_upload[:3]:
                print(f"   • {erreur}")
            if len(erreurs_upload) > 3:
                print(f"   • ... et {len(erreurs_upload) - 3} autres erreurs")

        return fichiers_uploades > 0

    except ImportError:
        print("❌ Erreur: Bibliothèque Office365 manquante")
        print("   Installez avec: pip install Office365-REST-Python-Client")
        return False

    except Exception as e:
        print(f"❌ Erreur SharePoint: {e}")
        return False